In [49]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

In [50]:
df = pd.read_csv("healthcare-dataset-stroke-data.csv")

In [51]:
df.head()


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [52]:
df = df.dropna()
df_saidas = df.pop("stroke")

df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked
...,...,...,...,...,...,...,...,...,...,...,...
5104,14180,Female,13.0,0,0,No,children,Rural,103.08,18.6,Unknown
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked


In [53]:
df_saidas.head()

0    1
2    1
3    1
4    1
5    1
Name: stroke, dtype: int64

In [54]:
df = pd.get_dummies(df)
df_saidas = df_saidas.replace(0, -1)



df

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,9046,67.0,0,1,228.69,36.6,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0
2,31112,80.0,0,1,105.92,32.5,0,1,0,0,...,0,1,0,0,1,0,0,0,1,0
3,60182,49.0,0,0,171.23,34.4,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
4,1665,79.0,1,0,174.12,24.0,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
5,56669,81.0,0,0,186.21,29.0,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,14180,13.0,0,0,103.08,18.6,1,0,0,1,...,0,0,0,1,1,0,1,0,0,0
5106,44873,81.0,0,0,125.20,40.0,1,0,0,0,...,0,0,1,0,0,1,0,0,1,0
5107,19723,35.0,0,0,82.99,30.6,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
5108,37544,51.0,0,0,166.29,25.6,0,1,0,0,...,0,1,0,0,1,0,0,1,0,0


In [55]:
df = df.drop("id", axis = 1)
df = df.drop("bmi", axis = 1)
df = df.drop("avg_glucose_level", axis = 1)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(df, df_saidas, train_size=0.5)


In [57]:
X_train.shape
y_train.shape

(2454,)

In [58]:
X_train = X_train.to_numpy()
X_train = X_train.T
y_train = y_train.to_numpy()


X_train  = X_train.astype(float)
y_train  = y_train.astype(float)


X_test = X_test.to_numpy()
X_test = X_test.T


y_test = y_test.to_numpy()


X_test  = X_test.astype(float)
y_test  = y_test.astype(float)



print(y_test.shape)

(2455,)


In [59]:
def accuracy(y_test, y_est):
    return np.mean(np.sign(y_test)==np.sign(y_est))


import autograd.numpy as np_   # Thinly-wrapped version of Numpy
from autograd import grad

def loss( parametros ):
    w, b, pontos, val = parametros
    est = w.T @ pontos + b
    mse = np_.mean( (est - val)**2)
    return mse

g = grad(loss)

pontos = X_train
alvos = y_train

w = np.random.randn( 19, 1)
b = np.random.randn()
alpha = 10**-4

for n in range(100000):

    grad_ = g( (w, b, pontos, alvos) )
    w -= alpha*grad_[0]
    b -= alpha*grad_[1]

print(w)
print(b)
print(len(grad_))

[[ 0.00507374]
 [ 0.05857075]
 [ 0.46653969]
 [-0.84357042]
 [-0.86814855]
 [-1.46927276]
 [ 0.07735286]
 [ 0.06945546]
 [ 0.29860498]
 [-0.34811702]
 [ 0.31710009]
 [ 0.24483069]
 [ 0.63158392]
 [-0.05168205]
 [-0.03972718]
 [-0.06507551]
 [ 0.07275508]
 [ 0.05276352]
 [ 0.01125386]]
-0.6973966171106546
4


In [60]:
def accuracy(y_test, y_est):
    return np.mean(np.sign(y_test)==np.sign(y_est))

In [61]:
y_est = w.T @ X_test + b
print(f"{accuracy(y_test, y_est)*100} %")

95.4378818737271 %
